In [121]:
import pandas as pd
import numpy as np

In [122]:
data = pd.read_csv("data.csv", parse_dates=['Data'])

In [123]:
data

,Data,Casa,Fora,Gols Casa,Gols Fora,Edição,Fase
0,2001-07-11,Equador,Chile,1.0,4.0,2001,Grupo A
1,2001-07-11,Colombia,Venezuela,2.0,0.0,2001,Grupo A
2,2001-07-14,Chile,Venezuela,1.0,0.0,2001,Grupo A
3,2001-07-14,Colombia,Equador,1.0,0.0,2001,Grupo A
4,2001-07-17,Equador,Venezuela,4.0,0.0,2001,Grupo A
...,...,...,...,...,...,...,...
207,2021-06-27,Venezuela,Peru,0.0,1.0,2021,Grupo B
208,2021-07-02,Peru,Paraguai,NaN,NaN,2021,Quartas
209,2021-07-02,Brasil,Chile,NaN,NaN,2021,Quartas
210,2021-07-03,Uruguai,Colombia,NaN,NaN,2021,Quartas


In [124]:
resultados = list()
for gc, gf in data[['Gols Casa', 'Gols Fora']].values:
    if gc > gf:
        resultados.append(1)
    elif gf > gc:
        resultados.append(-1)
    else:
        resultados.append(0)
data['resultados'] = resultados

In [125]:
ix = ["Edição","time"]
grupos = data[data['Fase'].str.contains("Grupo")]
newnames_casa = {"Casa": "time", -1: "derrota", 0:"empate", 1:"vitoria"}
newnames_fora = {"Fora": "time", -1: "vitoria", 0:"empate", 1:"derrota"}
grupos_casa = grupos.groupby(["Edição", "Casa", 'resultados']).size().unstack().fillna(0).reset_index().rename(columns=newnames_casa).set_index(ix)
grupos_fora = grupos.groupby(["Edição", "Fora", 'resultados']).size().unstack().fillna(0).reset_index().rename(columns=newnames_fora).set_index(ix)
partidas = grupos_casa.add(grupos_fora, fill_value = 0).reset_index()

In [126]:
ix = ["Edição","time"]
newnames_gcasa = {"Casa": "time", 'Gols Casa': "marcados", 'Gols Fora': "sofridos"}
gols_casa = grupos.groupby(["Edição", "Casa"])[['Gols Casa', 'Gols Fora']].sum().reset_index().rename(columns=newnames_gcasa).set_index(ix)
newnames_gfora = {"Fora": "time", 'Gols Casa': "sofridos", 'Gols Fora': "marcados"}
gols_fora = grupos.groupby(["Edição", "Fora"])[['Gols Casa', 'Gols Fora']].sum().reset_index().rename(columns=newnames_gfora).set_index(ix)
gols = gols_casa.add(gols_fora, fill_value=0).reset_index()

In [127]:
non_grupos = grupos = data[~data['Fase'].str.contains("Grupo")][['Edição','Casa', 'Fora', 'resultados']]

In [128]:
df = pd.merge(non_grupos, partidas, left_on=["Edição","Casa"], right_on=["Edição","time"], how='left')
df = pd.merge(df, partidas, left_on=["Edição","Fora"], right_on=["Edição","time"], how='left', suffixes=["_casa", "_fora"])
df = pd.merge(df, gols, left_on=["Edição","Casa"], right_on=["Edição","time"], how='left')
df = pd.merge(df, gols, left_on=["Edição","Fora"], right_on=["Edição","time"], how='left', suffixes=["_casa", "_fora"])

In [129]:
df.head()

,Edição,Casa,Fora,resultados,time_casa,derrota_casa,empate_casa,vitoria_casa,time_fora,derrota_fora,empate_fora,vitoria_fora,time_casa,marcados_casa,sofridos_casa,time_fora,marcados_fora,sofridos_fora
0,2001,Chile,Mexico,-1,Chile,1.0,0.0,2.0,Mexico,1.0,1.0,1.0,Chile,5.0,3.0,Mexico,1.0,1.0
1,2001,Costa Rica,Uruguai,-1,Costa Rica,0.0,1.0,2.0,Uruguai,1.0,1.0,1.0,Costa Rica,6.0,1.0,Uruguai,2.0,2.0
2,2001,Colombia,Peru,1,Colombia,0.0,0.0,3.0,Peru,1.0,1.0,1.0,Colombia,5.0,0.0,Peru,4.0,5.0
3,2001,Honduras,Brasil,1,Honduras,1.0,0.0,2.0,Brasil,1.0,0.0,2.0,Honduras,3.0,1.0,Brasil,5.0,2.0
4,2001,Mexico,Uruguai,1,Mexico,1.0,1.0,1.0,Uruguai,1.0,1.0,1.0,Mexico,1.0,1.0,Uruguai,2.0,2.0


In [146]:

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import log_loss, roc_auc_score, classification_report, f1_score

In [135]:
0.45783730158730157

0.45783730158730157

In [142]:
auc = list()
for e in [2004, 2007, 2011, 2015, 2016, 2019]:
    dftr = df[df['Edição'] < e]
    dfval = df[df['Edição'] == e]
    f = ['derrota_casa','empate_casa', 'vitoria_casa', 'derrota_fora',
       'empate_fora', 'vitoria_fora', 'marcados_casa',
       'sofridos_casa', 'marcados_fora', 'sofridos_fora']
    
    Xtr = dftr[f]
    ytr = dftr['resultados']
    Xval = dfval[f]
    yval = dfval['resultados']
    
    #print(dftr.shape)
    #print(dfval.shape)
    mdl = LogisticRegression(C=1, class_weight='balanced')
    #mdl = RandomForestClassifier(n_jobs=6, min_samples_leaf=2, n_estimators=100, random_state=0)
    mdl.fit(Xtr, ytr)
    
    
    p = mdl.predict_proba(Xval)
    p_ = mdl.predict(Xval)
    auc_ = roc_auc_score(yval,p,multi_class='ovr')
    f1 = 
    print("Ano: {} | LL: {} | AUC: {}".format(e, log_loss(yval, p), auc_))
    auc.append(auc_)
    #print(classification_report(yval, p_))
    print()
print(np.mean(auc))
    

Ano: 2004 | LL: 2.9514717901829335 | AUC: 0.22916666666666666

Ano: 2007 | LL: 1.1069725818244613 | AUC: 0.690873015873016

Ano: 2011 | LL: 3.32783622818049 | AUC: 0.25555555555555554

Ano: 2015 | LL: 1.3715442860035738 | AUC: 0.47142857142857136

Ano: 2016 | LL: 1.2713562077495835 | AUC: 0.6111111111111112

Ano: 2019 | LL: 1.4586912545073727 | AUC: 0.4888888888888889

0.45783730158730157


/Users/mariofilho/miniconda3/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/Users/mariofilho/miniconda3/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#l

In [143]:
p_

array([ 1,  0,  1,  0,  1,  0, -1,  1])

In [145]:
dfval

,Edição,Casa,Fora,resultados,time_casa,derrota_casa,empate_casa,vitoria_casa,time_fora,derrota_fora,empate_fora,vitoria_fora,time_casa,marcados_casa,sofridos_casa,time_fora,marcados_fora,sofridos_fora
48,2019,Brasil,Paraguai,0,Brasil,0.0,1.0,2.0,Paraguai,1.0,2.0,0.0,Brasil,8.0,0.0,Paraguai,3.0,4.0
49,2019,Venezuela,Argentina,-1,Venezuela,0.0,2.0,1.0,Argentina,1.0,1.0,1.0,Venezuela,3.0,1.0,Argentina,3.0,3.0
50,2019,Colombia,Chile,0,Colombia,0.0,0.0,3.0,Chile,1.0,0.0,2.0,Colombia,4.0,0.0,Chile,6.0,2.0
51,2019,Uruguai,Peru,0,Uruguai,0.0,1.0,2.0,Peru,1.0,1.0,1.0,Uruguai,7.0,2.0,Peru,3.0,6.0
52,2019,Brasil,Argentina,1,Brasil,0.0,1.0,2.0,Argentina,1.0,1.0,1.0,Brasil,8.0,0.0,Argentina,3.0,3.0
53,2019,Chile,Peru,-1,Chile,1.0,0.0,2.0,Peru,1.0,1.0,1.0,Chile,6.0,2.0,Peru,3.0,6.0
54,2019,Argentina,Chile,1,Argentina,1.0,1.0,1.0,Chile,1.0,0.0,2.0,Argentina,3.0,3.0,Chile,6.0,2.0
55,2019,Brasil,Peru,1,Brasil,0.0,1.0,2.0,Peru,1.0,1.0,1.0,Brasil,8.0,0.0,Peru,3.0,6.0
